In [1]:
# ===========================
# Biblio longform -> enrich (OPAC SBN -> Open Library -> LoC -> Internet Archive) -> output WIDE
# Single-cell Jupyter program
# - Robust OPAC SBN parsing (uses 'briefRecords')
# - Cleaner SBN query variants
# - Resumable (progress JSON + incremental long output)
# - Checkpoint reporting includes totals + per-source counts
# - DEBUG_SBN default OFF
# ===========================

import os, re, json, time
from difflib import SequenceMatcher
from urllib.parse import quote_plus

import pandas as pd
import requests

# ---------------------------
# CONFIG
# ---------------------------
INPUT_CSV    = "biblio_longform_all_records.csv"           # input longform: n_scheda,campo,valore
OUTPUT_LONG  = "biblio_longform_all_records_enriched.csv"  # enriched longform output
OUTPUT_WIDE  = "biblio_wide_records_enriched.csv"          # final wide output
PROGRESS_JSON = "biblio_enrich_progress.json"              # resumable run state

TOPK_PER_SOURCE = 12
TIMEOUT = 30
SLEEP_BETWEEN_CALLS = 0.25

MIN_SCORE_FOUND = 0.40      # accept best match if >= this
MIN_SCORE_KEEP  = 0.12      # keep candidate in pool if >= this

ALLOW_AUTHOR_ONLY_QUERIES = True
DEBUG_SBN = False           # default OFF (silences SBN per-query hints)

CONTACT_EMAIL = "pietro.terna@unito.it"
UA = f"biblio-enricher/3.1 (Jupyter; contact: {CONTACT_EMAIL})"

STANDARD_FIELDS = [
    "ext_source","ext_match_score","ext_title","ext_author","ext_year",
    "ext_publisher","ext_place","ext_language","ext_physical_desc",
    "ext_identifiers","ext_notes","ext_url"
]

# ---------------------------
# HTTP session
# ---------------------------
SESSION = requests.Session()
SESSION.headers.update({"User-Agent": UA})

def _sleep():
    time.sleep(SLEEP_BETWEEN_CALLS)

# ---------------------------
# Generic helpers
# ---------------------------
def norm(s: str) -> str:
    if s is None:
        return ""
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[“”\"'’`]", "", s)
    s = re.sub(r"[^0-9a-zàèéìòùçäëïöüßñ \-:/]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def sim(a: str, b: str) -> float:
    a, b = norm(a), norm(b)
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a, b).ratio()

def extract_year(s: str):
    if not s:
        return None
    m = re.search(r"(1[5-9]\d{2}|20\d{2})", str(s))
    return int(m.group(1)) if m else None

def normalize_year_from_data_pub(data_pub: str):
    # "COPYRIGHT 1966-68" -> 1966 ; "COPYRIGHT 1977" -> 1977
    return extract_year(data_pub)

def parse_series(collezion: str):
    if not collezion:
        return ("", "")
    s = str(collezion).strip()
    m = re.search(r"\bN\.?\s*(\d+)\b", s, flags=re.IGNORECASE)
    if m:
        no = m.group(1)
        series = re.sub(r"\bN\.?\s*\d+\b", "", s, flags=re.IGNORECASE).strip(" ,;-")
        return (series.strip(), no)
    return (s, "")

def pack_identifiers(d: dict) -> str:
    return json.dumps(d or {}, ensure_ascii=False)

def safe_list(v):
    if v is None:
        return []
    if isinstance(v, list):
        return v
    return [v]

def first_nonempty(vals):
    for v in vals:
        if v is None:
            continue
        s = str(v).strip()
        if s and s.lower() != "nan":
            return s
    return ""

def load_progress():
    if os.path.exists(PROGRESS_JSON):
        with open(PROGRESS_JSON, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"done_n_scheda": [], "last_book_number": 0}

def save_progress(prog):
    with open(PROGRESS_JSON, "w", encoding="utf-8") as f:
        json.dump(prog, f, ensure_ascii=False, indent=2)

# ---------------------------
# SBN-specific helpers
# ---------------------------
def sbn_clean_query(s: str) -> str:
    s = (s or "")
    s = re.sub(r"[\"'’`]", "", s)
    s = re.sub(r"[,.;:()\\[\\]{}<>|]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def sbn_extract_hits(js):
    """
    OPAC SBN mobilegw returns hits in 'briefRecords' (confirmed by your smoke test).
    Keep fallbacks for robustness.
    """
    if not isinstance(js, dict):
        return []
    for k in ["briefRecords", "records", "record", "results", "elenco", "items", "docs"]:
        v = js.get(k)
        if isinstance(v, list):
            return v
    for k in ["result", "response", "data"]:
        v = js.get(k)
        if isinstance(v, dict):
            for k2 in ["briefRecords", "records", "items", "docs", "results", "elenco"]:
                v2 = v.get(k2)
                if isinstance(v2, list):
                    return v2
    return []

def series_bonus(book, text):
    series = norm(book.get("series",""))
    s_no   = norm(book.get("series_no",""))
    t = norm(text)
    bonus = 0.0
    if series and series in t:
        bonus += 0.10
    if s_no and re.search(rf"\b{s_no}\b", t):
        bonus += 0.06
    return bonus

# ---------------------------
# Load input and group by book
# ---------------------------
df = pd.read_csv(INPUT_CSV)
need_cols = {"n_scheda","campo","valore"}
if not need_cols.issubset(df.columns):
    raise ValueError(f"CSV non nel formato atteso. Colonne trovate={list(df.columns)}; servono {sorted(need_cols)}")

df["n_scheda"] = df["n_scheda"].astype(int)
df["campo"] = df["campo"].astype(str)
df["valore"] = df["valore"].astype(str)

groups = list(df.groupby("n_scheda", sort=True))
n_books = len(groups)

def book_input_from_group(g: pd.DataFrame) -> dict:
    rec = {"n_scheda": int(g["n_scheda"].iloc[0])}

    def fv(field):
        vals = g.loc[g["campo"] == field, "valore"].tolist()
        return first_nonempty(vals)

    rec["autore"]    = fv("autore")
    rec["titolo"]    = fv("titolo")
    rec["editore"]   = fv("editore")
    rec["luogo_pub"] = fv("luogo_pub")
    rec["data_pub"]  = fv("data_pub")
    rec["cod_isbn"]  = fv("cod_isbn")
    rec["collezion"] = fv("collezion")

    rec["year"] = normalize_year_from_data_pub(rec["data_pub"]) or extract_year(rec["titolo"])
    series, series_no = parse_series(rec["collezion"])
    rec["series"] = series
    rec["series_no"] = series_no
    return rec

def build_queries(book: dict):
    titolo  = (book.get("titolo") or "").strip()
    autore  = (book.get("autore") or "").strip()
    editore = (book.get("editore") or "").strip()
    luogo   = (book.get("luogo_pub") or "").strip()
    year    = book.get("year")
    series  = (book.get("series") or "").strip()
    s_no    = (book.get("series_no") or "").strip()

    queries = []

    if titolo:
        q1 = " ".join([x for x in [titolo, autore, str(year) if year else ""] if x]).strip()
        q2 = " ".join([x for x in [titolo, autore] if x]).strip()
        q3 = " ".join([x for x in [titolo, autore, editore, str(year) if year else ""] if x]).strip()
        for q in [q1, q2, q3]:
            if q and q not in queries:
                queries.append(q)
        return queries

    # no title
    qA = " ".join([x for x in [autore, series, s_no, editore, luogo, str(year) if year else ""] if x]).strip()
    qB = " ".join([x for x in [autore, series, s_no] if x]).strip()
    qC = " ".join([x for x in [autore, series] if x]).strip()
    qD = " ".join([x for x in [autore, str(year) if year else "", editore] if x]).strip()

    for q in [qA, qB, qC, qD]:
        if q and q not in queries:
            queries.append(q)

    if ALLOW_AUTHOR_ONLY_QUERIES and autore and autore not in queries:
        queries.append(autore)

    return queries

# ---------------------------
# OPAC SBN (opacmobilegw)
# ---------------------------
def sbn_search_any(q: str, rows=TOPK_PER_SOURCE):
    url = f"https://opac.sbn.it/opacmobilegw/search.json?any={quote_plus(q)}&type=0&start=0&rows={rows}"
    r = SESSION.get(url, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def sbn_search_isbn(isbn: str, rows=TOPK_PER_SOURCE):
    url = f"https://opac.sbn.it/opacmobilegw/search.json?isbn={quote_plus(isbn)}&start=0&rows={rows}"
    r = SESSION.get(url, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def sbn_full(bid: str):
    url = f"https://opac.sbn.it/opacmobilegw/full.json?bid={quote_plus(bid)}"
    r = SESSION.get(url, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def _score_sbn_hits(book, hits):
    titolo_in = (book.get("titolo") or "").strip()
    autore_in = (book.get("autore") or "").strip()
    year_in   = book.get("year")

    cands = []
    for h in (hits or [])[:TOPK_PER_SOURCE]:
        title = h.get("titolo") or ""
        auth  = h.get("autorePrincipale") or ""
        pub   = h.get("pubblicazione") or ""
        bid   = h.get("codiceIdentificativo") or ""
        by = extract_year(pub)

        if titolo_in:
            score = 0.70 * sim(titolo_in, title) + 0.30 * sim(autore_in, auth)
        else:
            score = 0.85 * sim(autore_in, auth) + 0.15 * (0.2 if title else 0.0)

        score += series_bonus(book, f"{title} {pub}")

        if year_in and by and abs(year_in - by) <= 1:
            score += 0.06

        if score >= MIN_SCORE_KEEP:
            cands.append({"source":"OPAC SBN","score":score,"title":title,"author":auth,"pub":pub,"bid":bid,"raw":h})

    cands.sort(key=lambda x: x["score"], reverse=True)

    # full.json only for best 2
    for c in cands[:2]:
        if c.get("bid"):
            try:
                c["full"] = sbn_full(c["bid"])
            except Exception:
                c["full"] = None
    return cands

def sbn_candidates(book: dict):
    isbn = (book.get("cod_isbn") or "").strip()
    queries = build_queries(book)

    # ISBN-first
    if isbn:
        try:
            js = sbn_search_isbn(isbn)
            hits = sbn_extract_hits(js)
            if DEBUG_SBN:
                print(f"   SBN[isbn] numFound={js.get('numFound')} hits={len(hits)} isbn={isbn[:20]}")
            c = _score_sbn_hits(book, hits)
            if c:
                return c
        except Exception as e:
            if DEBUG_SBN:
                print(f"   SBN[isbn] ERROR: {type(e).__name__}")

    # Special cleaned query when title missing
    if not (book.get("titolo") or "").strip():
        q_special = sbn_clean_query(" ".join([
            book.get("autore",""),
            book.get("series",""),
            str(book.get("series_no","") or ""),
            book.get("editore",""),
            book.get("luogo_pub",""),
            str(book.get("year") or "")
        ]))
        if q_special:
            try:
                js = sbn_search_any(q_special)
                hits = sbn_extract_hits(js)
                if DEBUG_SBN:
                    print(f"   SBN[special] numFound={js.get('numFound')} hits={len(hits)} q='{q_special[:70]}'")
                c = _score_sbn_hits(book, hits)
                if c:
                    return c
            except Exception as e:
                if DEBUG_SBN:
                    print(f"   SBN[special] ERROR: {type(e).__name__}")

    # any= variants: original + cleaned
    for q in queries:
        if not q:
            continue
        for qv in [q, sbn_clean_query(q)]:
            if not qv:
                continue
            try:
                js = sbn_search_any(qv)
                hits = sbn_extract_hits(js)
                if DEBUG_SBN:
                    print(f"   SBN[any] numFound={js.get('numFound')} hits={len(hits)} q='{qv[:70]}'")
                c = _score_sbn_hits(book, hits)
                if c:
                    return c
            except Exception as e:
                if DEBUG_SBN:
                    print(f"   SBN[any] ERROR: {type(e).__name__} q='{qv[:40]}'")
                continue

    return []

# ---------------------------
# Open Library (q= only)
# ---------------------------
def ol_search_q(q: str, rows=TOPK_PER_SOURCE):
    params = {"q": q, "limit": rows}
    r = SESSION.get("https://openlibrary.org/search.json", params=params, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def ol_candidates(book: dict):
    isbn = (book.get("cod_isbn") or "").strip()
    queries = []
    if isbn:
        queries.append(f"isbn:{isbn}")
    queries.extend(build_queries(book))

    titolo_in = (book.get("titolo") or "").strip()
    autore_in = (book.get("autore") or "").strip()
    year_in   = book.get("year")

    for q in queries:
        if not q:
            continue
        try:
            js = ol_search_q(q)
        except Exception:
            continue

        docs = js.get("docs", [])[:TOPK_PER_SOURCE]
        cands = []
        for d in docs:
            title = d.get("title") or ""
            auth  = (d.get("author_name") or [""])[0]
            year  = d.get("first_publish_year")
            pub   = (d.get("publisher") or [""])[0] if isinstance(d.get("publisher"), list) else (d.get("publisher") or "")
            place = (d.get("publish_place") or [""])[0] if isinstance(d.get("publish_place"), list) else (d.get("publish_place") or "")
            lang  = (d.get("language") or [""])[0] if isinstance(d.get("language"), list) else (d.get("language") or "")

            if titolo_in:
                score = 0.70 * sim(titolo_in, title) + 0.30 * sim(autore_in, auth)
            else:
                score = 0.85 * sim(autore_in, auth) + 0.15 * (0.2 if title else 0.0)

            score += series_bonus(book, f"{title} {pub} {place}")

            if year_in and year and abs(year_in - int(year)) <= 1:
                score += 0.05

            if score >= MIN_SCORE_KEEP:
                cands.append({"source":"Open Library","score":score,"title":title,"author":auth,"year":year,
                              "publisher":pub,"place":place,"language":lang,"raw":d})

        cands.sort(key=lambda x: x["score"], reverse=True)
        if cands:
            return cands

    return []

# ---------------------------
# Library of Congress
# ---------------------------
def loc_search(q: str, rows=TOPK_PER_SOURCE):
    params = {"fo":"json", "q": q, "c": rows}
    r = SESSION.get("https://www.loc.gov/books/", params=params, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def loc_candidates(book: dict):
    queries = build_queries(book)
    titolo_in = (book.get("titolo") or "").strip()
    autore_in = (book.get("autore") or "").strip()
    year_in   = book.get("year")

    for q in queries:
        if not q:
            continue
        try:
            js = loc_search(q)
        except Exception:
            continue

        results = js.get("results", [])[:TOPK_PER_SOURCE]
        cands = []
        for it in results:
            title = it.get("title") or ""
            auth_list = it.get("contributor") or it.get("creator") or []
            auth = auth_list[0] if isinstance(auth_list, list) and auth_list else ""
            date = it.get("date") or ""
            year = extract_year(date)
            loc_id = it.get("id") or ""

            if titolo_in:
                score = 0.70 * sim(titolo_in, title) + 0.30 * sim(autore_in, auth)
            else:
                score = 0.85 * sim(autore_in, auth) + 0.15 * (0.2 if title else 0.0)

            score += series_bonus(book, f"{title} {date}")

            if year_in and year and abs(year_in - year) <= 1:
                score += 0.05

            if score >= MIN_SCORE_KEEP:
                cands.append({"source":"Library of Congress","score":score,"title":title,"author":auth,"date":date,"id":loc_id,"raw":it})

        cands.sort(key=lambda x: x["score"], reverse=True)
        if cands:
            return cands

    return []

# ---------------------------
# Internet Archive
# ---------------------------
def ia_advanced_search(title: str, author: str, q_extra: str = "", rows=TOPK_PER_SOURCE):
    q = []
    if title:
        q.append(f'title:("{title}")')
    if author:
        q.append(f'creator:("{author}")')
    if q_extra:
        q.append(q_extra)
    q.append('(mediatype:texts OR collection:opensource_texts)')
    query = " AND ".join([x for x in q if x])

    params = {
        "q": query,
        "fl[]": ["identifier", "title", "creator", "date", "publisher", "language"],
        "rows": rows,
        "page": 1,
        "output": "json"
    }
    r = SESSION.get("https://archive.org/advancedsearch.php", params=params, timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def ia_metadata(identifier: str):
    r = SESSION.get(f"https://archive.org/metadata/{identifier}", timeout=TIMEOUT)
    _sleep()
    r.raise_for_status()
    return r.json()

def ia_candidates(book: dict):
    titolo_in = (book.get("titolo") or "").strip()
    autore_in = (book.get("autore") or "").strip()
    year_in   = book.get("year")
    queries = build_queries(book)

    cands_all = []
    for q in queries[:3]:
        try:
            js = ia_advanced_search(titolo_in if titolo_in else "", autore_in if autore_in else "", q_extra=q)
        except Exception:
            continue

        docs = (js.get("response", {}).get("docs", []) or [])[:TOPK_PER_SOURCE]
        for d in docs:
            title = d.get("title") or ""
            creator = d.get("creator")
            auth = creator[0] if isinstance(creator, list) and creator else (creator or "")
            year = extract_year(d.get("date"))
            ident = d.get("identifier")

            if titolo_in:
                score = 0.70 * sim(titolo_in, title) + 0.30 * sim(autore_in, auth)
            else:
                score = 0.85 * sim(autore_in, auth) + 0.15 * (0.2 if title else 0.0)

            score += series_bonus(book, f"{title} {d.get('publisher','')}")

            if year_in and year and abs(year_in - year) <= 1:
                score += 0.05

            if score >= MIN_SCORE_KEEP:
                cands_all.append({"source":"Internet Archive","score":score,"title":title,"author":auth,"year":year,
                                  "identifier":ident,"raw":d})

        if cands_all:
            break

    cands_all.sort(key=lambda x: x["score"], reverse=True)

    if cands_all and cands_all[0].get("identifier"):
        try:
            cands_all[0]["full"] = ia_metadata(cands_all[0]["identifier"])
        except Exception:
            cands_all[0]["full"] = None

    return cands_all

# ---------------------------
# Candidate -> ext_* mapping
# ---------------------------
def from_sbn(best: dict) -> dict:
    full = best.get("full") or {}
    return {
        "ext_source": "OPAC SBN",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": full.get("titolo") or best.get("title") or "",
        "ext_author": full.get("autorePrincipale") or best.get("author") or "",
        "ext_year": str(extract_year(full.get("pubblicazione")) or extract_year(best.get("pub")) or "" or ""),
        "ext_publisher": "",
        "ext_place": "",
        "ext_language": full.get("linguaPubblicazione") or "",
        "ext_physical_desc": full.get("descrizioneFisica") or "",
        "ext_identifiers": pack_identifiers({"bid": best.get("bid")}),
        "ext_notes": " | ".join(safe_list(full.get("note"))) if full.get("note") else "",
        "ext_url": f"https://opac.sbn.it/opacmobilegw/full.json?bid={quote_plus(best.get('bid',''))}" if best.get("bid") else ""
    }

def from_ol(best: dict) -> dict:
    d = best.get("raw") or {}
    identifiers = {}
    for k in ["isbn", "oclc", "lccn"]:
        if k in d and d[k]:
            identifiers[k] = d[k][:10] if isinstance(d[k], list) else d[k]
    return {
        "ext_source": "Open Library",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": best.get("title") or "",
        "ext_author": best.get("author") or "",
        "ext_year": str(best.get("year") or ""),
        "ext_publisher": best.get("publisher") or "",
        "ext_place": best.get("place") or "",
        "ext_language": best.get("language") or "",
        "ext_physical_desc": "",
        "ext_identifiers": pack_identifiers(identifiers),
        "ext_notes": "",
        "ext_url": f"https://openlibrary.org{d.get('key','')}" if d.get("key") else ""
    }

def from_loc(best: dict) -> dict:
    return {
        "ext_source": "Library of Congress",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": best.get("title") or "",
        "ext_author": best.get("author") or "",
        "ext_year": str(extract_year(best.get("date") or "") or ""),
        "ext_publisher": "",
        "ext_place": "",
        "ext_language": "",
        "ext_physical_desc": "",
        "ext_identifiers": pack_identifiers({"loc_id": best.get("id")}),
        "ext_notes": "",
        "ext_url": best.get("id") or ""
    }

def from_ia(best: dict) -> dict:
    full = best.get("full") or {}
    md = full.get("metadata") or {}

    def md1(k):
        v = md.get(k, "")
        if isinstance(v, list):
            return v[0] if v else ""
        return v or ""

    return {
        "ext_source": "Internet Archive",
        "ext_match_score": f"{best.get('score',0):.3f}",
        "ext_title": md1("title") or best.get("title") or "",
        "ext_author": md1("creator") or best.get("author") or "",
        "ext_year": str(extract_year(md1("date")) or best.get("year") or ""),
        "ext_publisher": md1("publisher"),
        "ext_place": md1("publishplace"),
        "ext_language": md1("language"),
        "ext_physical_desc": md1("description"),
        "ext_identifiers": pack_identifiers({"identifier": best.get("identifier")}),
        "ext_notes": md1("notes"),
        "ext_url": f"https://archive.org/details/{best.get('identifier')}" if best.get("identifier") else ""
    }

def choose_best_candidate(book: dict):
    sbn = sbn_candidates(book)
    if sbn and sbn[0]["score"] >= MIN_SCORE_FOUND:
        return sbn[0], from_sbn(sbn[0])

    ol = ol_candidates(book)
    if ol and ol[0]["score"] >= MIN_SCORE_FOUND:
        return ol[0], from_ol(ol[0])

    loc = loc_candidates(book)
    if loc and loc[0]["score"] >= MIN_SCORE_FOUND:
        return loc[0], from_loc(loc[0])

    ia = ia_candidates(book)
    if ia and ia[0]["score"] >= MIN_SCORE_FOUND:
        return ia[0], from_ia(ia[0])

    return None, {
        "ext_source":"NOT_FOUND","ext_match_score":"0.000",
        "ext_title":"","ext_author":"","ext_year":"",
        "ext_publisher":"","ext_place":"","ext_language":"",
        "ext_physical_desc":"","ext_identifiers":pack_identifiers({}),
        "ext_notes":"","ext_url":""
    }

# ---------------------------
# Enrichment run (resumable)
# ---------------------------
prog = load_progress()
done = set(int(x) for x in prog.get("done_n_scheda", []))

if os.path.exists(OUTPUT_LONG):
    out_df = pd.read_csv(OUTPUT_LONG)
    out_df["n_scheda"] = out_df["n_scheda"].astype(int)
    out_df["campo"] = out_df["campo"].astype(str)
    out_df["valore"] = out_df["valore"].astype(str)
else:
    out_df = df.copy()

# Track ext fields already present (avoid duplicates)
already_ext = set()
mask_ext = out_df["campo"].astype(str).str.startswith("ext_")
for r in out_df.loc[mask_ext, ["n_scheda","campo"]].itertuples(index=False):
    already_ext.add((int(r.n_scheda), str(r.campo)))

added_rows = []

found_count = 0
notfound_count = 0
found_by_source = {
    "OPAC SBN": 0,
    "Open Library": 0,
    "Library of Congress": 0,
    "Internet Archive": 0
}

for book_number, (n_scheda, g) in enumerate(groups, start=1):
    print(f"[{book_number}/{n_books}] n_scheda={n_scheda}")
    if int(n_scheda) in done:
        continue

    book = book_input_from_group(g)
    cand, ext = choose_best_candidate(book)
    src = ext.get("ext_source")

    if src != "NOT_FOUND":
        found_count += 1
        if src in found_by_source:
            found_by_source[src] += 1
    else:
        notfound_count += 1

    for k in STANDARD_FIELDS:
        key = (int(n_scheda), k)
        if key in already_ext:
            continue
        added_rows.append({"n_scheda": int(n_scheda), "campo": k, "valore": ext.get(k,"")})
        already_ext.add(key)

    # checkpoint state
    done.add(int(n_scheda))
    prog["done_n_scheda"] = sorted(done)
    prog["last_book_number"] = book_number
    save_progress(prog)

    # write periodically
    if book_number % 10 == 0:
        if added_rows:
            out_df = pd.concat([out_df, pd.DataFrame(added_rows)], ignore_index=True)
            added_rows = []
        out_df.to_csv(OUTPUT_LONG, index=False)
        print(
            f"Checkpoint: {OUTPUT_LONG} | "
            f"found={found_count} not_found={notfound_count} | "
            f"SBN={found_by_source['OPAC SBN']} "
            f"OL={found_by_source['Open Library']} "
            f"LoC={found_by_source['Library of Congress']} "
            f"IA={found_by_source['Internet Archive']}"
        )

# final write long
if added_rows:
    out_df = pd.concat([out_df, pd.DataFrame(added_rows)], ignore_index=True)
out_df.to_csv(OUTPUT_LONG, index=False)

print(
    f"Enrichment completato. LONG: {OUTPUT_LONG} | "
    f"found={found_count} not_found={notfound_count} | "
    f"SBN={found_by_source['OPAC SBN']} "
    f"OL={found_by_source['Open Library']} "
    f"LoC={found_by_source['Library of Congress']} "
    f"IA={found_by_source['Internet Archive']}"
)

# ---------------------------
# LONG -> WIDE (robust, avoid n_scheda collision)
# ---------------------------
df_long = out_df.copy()
df_long["valore"] = df_long["valore"].fillna("").astype(str)

# Avoid collision: field named 'n_scheda' inside 'campo' becomes 'n_scheda_field'
df_long.loc[df_long["campo"] == "n_scheda", "campo"] = "n_scheda_field"

# Aggregate duplicates per (n_scheda, campo)
agg = (df_long
       .groupby(["n_scheda","campo"])["valore"]
       .agg(lambda s: " | ".join([v.strip() for v in s if isinstance(v,str) and v.strip()]))
       .reset_index()
      )

df_wide = agg.pivot(index="n_scheda", columns="campo", values="valore").reset_index()

# Front-load key columns if present
front = ["n_scheda","titolo","autore","data_pub","luogo_pub","editore","collezion",
         "ext_source","ext_match_score","ext_title","ext_author","ext_year","ext_url","ext_identifiers","n_scheda_field"]
cols = [c for c in front if c in df_wide.columns] + [c for c in df_wide.columns if c not in front]
df_wide = df_wide[cols]

df_wide.to_csv(OUTPUT_WIDE, index=False)
print(f"WIDE creato: {OUTPUT_WIDE} | righe={len(df_wide)} colonne={len(df_wide.columns)}")


[1/1000] n_scheda=41186
[2/1000] n_scheda=41187
[3/1000] n_scheda=41188
[4/1000] n_scheda=41189
[5/1000] n_scheda=41190
[6/1000] n_scheda=41191
[7/1000] n_scheda=41192
[8/1000] n_scheda=41193
[9/1000] n_scheda=41194
[10/1000] n_scheda=41195
Checkpoint: biblio_longform_all_records_enriched.csv | found=7 not_found=3 | SBN=7 OL=0 LoC=0 IA=0
[11/1000] n_scheda=41196
[12/1000] n_scheda=41197
[13/1000] n_scheda=41198
[14/1000] n_scheda=41199
[15/1000] n_scheda=41200
[16/1000] n_scheda=41201
[17/1000] n_scheda=41202
[18/1000] n_scheda=41203
[19/1000] n_scheda=41204
[20/1000] n_scheda=41205
Checkpoint: biblio_longform_all_records_enriched.csv | found=17 not_found=3 | SBN=14 OL=3 LoC=0 IA=0
[21/1000] n_scheda=41206
[22/1000] n_scheda=41207
[23/1000] n_scheda=41208
[24/1000] n_scheda=41209
[25/1000] n_scheda=41210
[26/1000] n_scheda=41211
[27/1000] n_scheda=41212
[28/1000] n_scheda=41213
[29/1000] n_scheda=41214
[30/1000] n_scheda=41215
Checkpoint: biblio_longform_all_records_enriched.csv | foun